# Search and Download OPERA RTC-S1 Data from ASF

<div class="alert alert-info" style="display: flex; align-items: center; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%;">
    <a href="https://github.com/ASFOpenSARlab/opensarlab_OPERA-CSLC_Recipe_Book/issues">
      <img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/logos/github_issues.png" alt="GitHub logo over the word Issues" style="width: 100%;">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Did you find a bug? Do you have a feature request?</b>
    <br/>
    Explore GitHub Issues on this Jupyter Book's GitHub repository. Find solutions, add to the discussion, or start a new bug report or feature request: <a href="https://github.com/ASFOpenSARlab/opensarlab_OPERA-RTC-S1_Recipe_Book/issues">opensarlab_OPERA-RTC-S1_Recipe_Book Issues</a>
  </div>
</div>

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%; margin-right: 10px;">
    <a href="mailto:uso@asf.alaska.edu">
      <img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/logos/ASF_support_logo.png" alt="ASF logo" style="width: 100%">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Have a question related to SAR, OPERA RTCs, or ASF data access?</b>
    <br/>
    Contact ASF User Support: <a href="mailto:uso@asf.alaska.edu">uso@asf.alaska.edu</a>
  </div>
</div>

## 1. Import needed Python package

In [ ]:
from getpass import getpass
from pathlib import Path
from pprint import pprint
import re

import asf_search as disco
from asf_search.download.file_download_type import FileDownloadType
import opensarlab_lib as osl

## 2. Authenticate with asf_search and start a session

**Gather credentials for authentication with Earth Data Login**

In [ ]:
username = input('Enter your EDL username')
password = getpass('Enter your EDL password')

**Start an asf_search session**

In [ ]:
try:
    user_pass_session = disco.ASFSession().auth_with_creds(username, password)
except disco.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

## 3. Create a directory in which to download the RTCs

**Create a data directory alongside the Jupyter Book containing this notebook** 

In [ ]:
name = input('Enter the name of a directory to create, which will hold your downloaded RTCs')
data_dir = Path.home() / name
data_dir.mkdir(exist_ok=True)
print(f"Data Directory: {data_dir}")

## 4. Search for OPERA RTC-S1 products

**Option 1: List Search**

- Search by OPERA product ID

- Identify OPERA IDs with an [OPERA-S1 Geographic Search on Vertex](https://search.asf.alaska.edu/#/?maxResults=250&dataset=OPERA-S1)


In [ ]:
product_list = [
    'OPERA_L2_RTC-S1_T173-370303-IW1_20231006T134409Z_20231007T132700Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T173-370304-IW1_20231006T134412Z_20231007T132700Z_S1A_30_v1.0',
    'OPERA_L2_RTC-S1_T173-370305-IW1_20231006T134415Z_20231007T132700Z_S1A_30_v1.0',
    ]
list_results = disco.granule_search(product_list)
# print(list_results)

**Option 2: Geographic Search**
- Search by intersection with an area of interest
  - Define in WKT as a POINT or POLYGON
    - `'intersectsWith': 'POINT(-117.4231 35.7597)'`
    - `'intersectsWith': 'POLYGON((-114.9066 35.2997,-114.159 35.2997,-114.159 35.8598,-114.9066 35.8598,-114.9066 35.2997))'`

In [ ]:
options = {
    'intersectsWith': 'POINT(-114.4556 36.1562)',
    'dataset': 'OPERA-S1',
    'start': '2023-11-01T00:00:00Z',
    'end': '2024-05-01T00:00:00Z',
    # 'flightDirection': 'ASCENDING',
    'processingLevel': ['RTC'],
    'maxResults': '1000'
}

results = disco.search(**options)
burst_ids = list(set([r.properties['operaBurstID'] for r in results]))
print(f"Found {len(results)} RTCs with {len(burst_ids)} burst IDs")

## 5. Filter the Results

**Select the burst ID to download**

In [ ]:
# We recommend T173_370304_IW1 here
burst_select = osl.select_parameter(burst_ids[::-1], 'Select the RTC burst ID you wish to download')
display(burst_select)

In [ ]:
bursts = [burst_select.value]
selected_results = [r for r in results if r.properties['operaBurstID'] in bursts]
selected_results

**Create directories for each burst**

In [ ]:
for b in bursts:
    (data_dir/b).mkdir(exist_ok=True)

**When >1 RTC are found for a burst/acquisition date, select the most recently processed bursts**

The geographic search may return current and superceded data


In [ ]:
filtered_bursts = dict()
acquisition_date_regex = r'(?<=OPERA_L2_RTC-S1_)T\d{3}-\d{6}-IW\d_\d{8}T\d{6}Z(?=_\d{8}T\d{6}Z)'
process_date_regex = r'(?<=OPERA_L2_RTC-S1_T\d{3}-\d{6}-IW\d_\d{8}T\d{6}Z_)\d{8}T\d{6}Z'

for b in selected_results:
    rtc_id = b.properties['fileID']
    try:
        id_date = re.search(acquisition_date_regex, rtc_id).group(0)
        try:
            # for bursts that only differ by processing date, we can use a simple relational comparison
            if filtered_bursts[id_date].properties['fileID'] < rtc_id:
                filtered_bursts[id_date] = b
        except KeyError:
            filtered_bursts[id_date] = b
    except AttributeError:
        raise Exception(f'Acquisition not found in RTC ID: {str(b)}')
        
filtered_bursts = list(filtered_bursts.values())   

for b in filtered_bursts:
    print(b.properties['fileID'])

## 6. Download Data

**Download the RTCs**

In [ ]:
for b in filtered_bursts:
    rtc_dir = data_dir/b.properties['operaBurstID']/b.properties['fileID']
    rtc_dir.mkdir(exist_ok=True)
    disco.download_urls(urls=[b.properties['url'], *b.properties['additionalUrls']], path=rtc_dir, session=user_pass_session)

**Download the static layers**

In [ ]:
for b in bursts:
    options = {
        'dataset': 'OPERA-S1',
        'operaBurstID': b,
        'processingLevel': ['RTC-STATIC'],
        'maxResults': '1000'
    }
    results = disco.search(**options)
    if len(results) > 0:
        static_dir = data_dir/b/f"{b}_STATIC"
        static_dir.mkdir(exist_ok=True)
        # results[0].download(static_dir, session=user_pass_session, fileType=FileDownloadType.ALL_FILES)
        disco.download_urls(
            path=static_dir, 
            urls=[results[0].properties['url'], *results[0].properties['additionalUrls']], 
            session=user_pass_session)

## 7. (Optional) Merge neighboring RTCs

In [ ]:
# Using the list search results from Section 4
merge_dir = data_dir / 'merge'
merge_dir.mkdir(exist_ok=True)

for b in list_results:
    rtc_dir = merge_dir / b.properties['fileID']
    rtc_dir.mkdir(exist_ok=True)
    disco.download_urls(urls=[b.properties['url'], *b.properties['additionalUrls']], path=rtc_dir, session=user_pass_session)

In [ ]:
from osgeo import gdal
import osgeo_utils.gdal_merge as gdal_merge

gdal.UseExceptions()


for file_type in ['VV', 'VH', 'mask']:
    output_file_path = merge_dir / f'merged_{file_type}.tif'
    input_file_paths = list(merge_dir.glob(f'OPERA_L2_RTC-S1*/*{file_type}.tif'))

    parameters = ['', '-o', output_file_path] + input_file_paths + ['-co', 'COMPRESS=LZW']
    gdal_merge.main(parameters)